In [212]:
import luigi
import pandas as pd
import os
import logging



Création du dossier pour stocker les données nétoyer

In [213]:
# Vérifiez si le dossier n'existe pas déjà

import shutil


dossier = 'CleanData'

if not os.path.exists(dossier):
    # Créez le dossier
    os.makedirs(dossier, mode=0o777)
    print("Dossier créé avec succès.")


In [214]:
class ExtractCandidats(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/candidats.csv")

    def run(self):
                
        df = pd.read_csv(self.file_path, sep=";")
       
        df = df.loc[:, ["Prenom", "Nom"]]
        df = df.drop_duplicates()
        
        df.to_csv(self.output().path, index=False)

In [215]:
import sys

class ReadDataCanton(luigi.Task):
    file_path = luigi.Parameter()

    def requires(self):
        return [ExtractCandidats(file_path="./RawData/canton_t1_2017.csv")]

    def output(self):
        return luigi.LocalTarget("./CleanData/canton.csv")

    def run(self):
                
        df = pd.read_csv(self.file_path, sep=";")

        #df_informations_base = df.loc[:, df.columns[0:18]]
        cantons = pd.DataFrame(columns=df.columns[0:23])
        

        for index, row in df.iterrows():
            #print(f"index : {index} \n row : {row[0:18]} \n\n")
            new_row = pd.Series(row[0:23], index=df.columns[0:23])
            cantons.loc[len(cantons)] = new_row


        display(cantons)

        cantons.to_csv(self.output().path, index=False)

In [216]:
class ReadDataCDSP(luigi.Task):
    file_path = luigi.Parameter()

    def output(self):
        return luigi.LocalTarget("./CleanData/output.csv")

    def run(self):
                        
        df = pd.read_csv(self.file_path, sep=";")
        # Faites quelque chose avec le dataframe, par exemple, l'afficher

        df.to_csv(self.output().path, index=False)

In [217]:
class ReadAllData(luigi.Task):
    def requires(self):
        return [ReadDataCanton(file_path="./RawData/canton_t1_2017.csv"), 
                ReadDataCDSP(file_path="./RawData/cdsp_presi2012t2_circ.csv")]
    
    def run(self):
        print("lancement")

    def output(self):
        return luigi.LocalTarget('result.txt')

In [218]:

config = luigi.configuration.get_config()
config.set('core', 'no_lock', 'True')

dossier = "./CleanData/"

# Parcourir tous les fichiers du dossier
for fichier in os.listdir(dossier):
    chemin_fichier = os.path.join(dossier, fichier)
    # Supprimer le fichier
    os.remove(chemin_fichier)

luigi.build([ReadAllData()], local_scheduler=False, no_lock=True)

DEBUG: Checking if ReadAllData() is complete
DEBUG: Checking if ReadDataCanton(file_path=./RawData/canton_t1_2017.csv) is complete
DEBUG: Checking if ReadDataCDSP(file_path=./RawData/cdsp_presi2012t2_circ.csv) is complete
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   PENDING
INFO: Informed scheduler that task   ReadDataCDSP___RawData_cdsp_p_0f11afe572   has status   PENDING
DEBUG: Checking if ExtractCandidats(file_path=./RawData/canton_t1_2017.csv) is complete
INFO: Informed scheduler that task   ReadDataCanton___RawData_canton_95a4352c04   has status   PENDING
INFO: Informed scheduler that task   ExtractCandidats___RawData_canton_95a4352c04   has status   PENDING
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 4
INFO: [pid 660] Worker Worker(salt=039911701, workers=1, host=DESKTOP-5342GOQ, username=Pendi, pid=660) running   ReadDataCDSP(file_path=./RawData/cdsp_presi2012t2_circ.cs

,Code du departement,Libelle du departement,Code du canton,Libelle du canton,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,...,% Nuls/Ins,% Nuls/Vot,Exprimes,% Exp/Ins,% Exp/Vot,N°Panneau,Sexe,Nom,Prenom,Voix
0,1,Ain,1,Amberieu-en-Bugey,19922,3909,"19,62",16013,"80,38",353,...,"0,62","0,77",15536,"77,98","97,02",2,F,LE PEN,Marine,4408
1,1,Ain,2,Attignat,17620,3029,"17,19",14591,"82,81",326,...,"0,5","0,6",14177,"80,46","97,16",2,F,LE PEN,Marine,3949
2,1,Ain,3,Bellegarde-sur-Valserine,12667,3051,"24,09",9616,"75,91",200,...,"0,53","0,7",9349,"73,81","97,22",2,F,LE PEN,Marine,2256
3,1,Ain,4,Belley,17644,3335,"18,9",14309,"81,1",266,...,"0,69","0,85",13922,"78,91","97,3",2,F,LE PEN,Marine,3615
4,1,Ain,5,Bourg-en-Bresse-1,20410,5123,"25,1",15287,"74,9",244,...,"0,33","0,44",14975,"73,37","97,96",3,M,MACRON,Emmanuel,3672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,ZP,Polynesie francaise,6,AUSTRALES (fictif),5636,2361,"41,89",3275,"58,11",84,...,"1,19","2,05",3124,"55,43","95,39",11,M,FILLON,Francois,1428
2086,ZS,Saint-Pierre-et-Miquelon,1,Canton fictif,4966,2238,"45,07",2728,"54,93",70,...,"0,52","0,95",2632,53,"96,48",9,M,MeLENCHON,Jean-Luc,933
2087,ZW,Wallis et Futuna,1,1er CANTON,8464,3053,"36,07",5411,"63,93",35,...,"0,33","0,52",5348,"63,19","98,84",3,M,MACRON,Emmanuel,1630
2088,ZX,Saint-Martin/Saint-Barthelemy,1,1er Canton,25385,17202,"67,76",8183,"32,24",211,...,"0,42","1,31",7865,"30,98","96,11",11,M,FILLON,Francois,2518


INFO: [pid 660] Worker Worker(salt=039911701, workers=1, host=DESKTOP-5342GOQ, username=Pendi, pid=660) done      ReadDataCanton(file_path=./RawData/canton_t1_2017.csv)
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ReadDataCanton___RawData_canton_95a4352c04   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 1
INFO: [pid 660] Worker Worker(salt=039911701, workers=1, host=DESKTOP-5342GOQ, username=Pendi, pid=660) running   ReadAllData()
INFO: [pid 660] Worker Worker(salt=039911701, workers=1, host=DESKTOP-5342GOQ, username=Pendi, pid=660) done      ReadAllData()
DEBUG: 1 running tasks, waiting for next task to finish
INFO: Informed scheduler that task   ReadAllData__99914b932b   has status   DONE
DEBUG: Asking scheduler for work...
DEBUG: Done
DEBUG: There are no more tasks to run at this time
INFO: Worker Worker(salt=039911701, workers=1, host=DESKTOP-5342GOQ, username=Pendi, pid=660) was stopped. Shutting down Ke

lancement


True